In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

### wikipedia tool

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki_tool.name

'wikipedia'

## Building RAG

#### building retriever tool form web content

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader=WebBaseLoader("https://python.langchain.com/v0.1/docs/modules/tools/")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retreiver=vectordb.as_retriever()
retreiver



VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B411826B60>)

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retreiver,"lanchain tools",
                      "search for any question about tools in langchian ")

In [6]:
retriever_tool.name

'lanchain tools'

### ARXIV Tool

arxiv- website where all the research paper have been uploaded

In [7]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [8]:
tools=[wiki_tool,retriever_tool,arxiv_tool]

In [9]:
#prompts
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [10]:
from langchain_community.llms import Ollama
#load ollama llama3 llm model
llm=Ollama(model='llama3:latest')
# Bind tools to LLM
llm_with_tools=llm.bind_tools()

AttributeError: 'Ollama' object has no attribute 'bind_tools'

In [11]:
#Agents
#Create an agent that uses tools.
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm,tools,prompt)

ValueError: This function requires a .bind_tools method be implemented on the LLM.

In [12]:
#agents executer
from langchain.agents import AgentExecutor
AgentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

NameError: name 'agent' is not defined

In [13]:
agent_executor.invoke({"input": "what is LangChain tools?"})

NameError: name 'agent_executor' is not defined

# ohh no ChatOllama  don't yet implement bind_tools.